In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('C:/Users/Hoe/Desktop/Learning/DataTalksClub_MLOPS/data/fhv_tripdata_2021-01.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

### Question 1

Run this notebook for the February 2021 FVH data.

What's the mean predicted duration for this dataset?

In [24]:
print(f'Answer Question 1 - mean predicted duration: {y_pred.mean()}')

Answer Question 1 - mean predicted duration: 16.2472533682457


### Question 2

What's the size of the output file?

In [25]:
df['prediction'] = y_pred

In [26]:
df['ride_id'] = '{year:04d}/{month:02d}_' + df.index.astype('str')

In [27]:
df_result = df[['ride_id', 'prediction']]

In [28]:
df_result

,ride_id,prediction
0,{year:04d}/{month:02d}_0,23.052085
1,{year:04d}/{month:02d}_1,23.052085
3,{year:04d}/{month:02d}_3,14.105704
4,{year:04d}/{month:02d}_4,13.817206
5,{year:04d}/{month:02d}_5,14.878253
...,...,...
1154107,{year:04d}/{month:02d}_1154107,12.979352
1154108,{year:04d}/{month:02d}_1154108,10.596450
1154109,{year:04d}/{month:02d}_1154109,16.613957
1154110,{year:04d}/{month:02d}_1154110,11.137382


In [29]:
output_file = 'output.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [30]:
import os
os.path.getsize("output.parquet")

38802666

### Question 3

Now let's turn the notebook into a script.

Which command you need to execute for that?

In [31]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1499 bytes to starter.py


### Question 4

Now let's put everything into a virtual environment. We'll use pipenv for that.
Install all the required libraries. Pay attention to the Scikit-Learn version: check the starter notebook for details.
After installing the libraries, pipenv creates two files: Pipfile and Pipfile.lock. The Pipfile.lock file keeps the hashes of the dependencies we use for the virtual env.
What's the first hash for the Scikit-Learn dependency?

"scikit-learn": {
    "hashes": [
        "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b",

08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b